## Handle Failing Tasks

All applications can fail, often for reasons out of control of the user. A Task is no different, it can fail as well. Many non-trivial application will need to have a way to handle failing tasks – detecting the failure is the first and necessary step to do so, and RP makes that part easy: RP’s task state model defines that a failing task will immediately go into FAILED state, and that state information is available as task.state property.

The task also has the task.stderr property available for further inspection into causes of the failure – that will only be available though if the task did reach the EXECUTING state in the first place.


In [ ]:
%load_ext dotenv
%dotenv ../../../.env

We start by importing the radical.pilot module and initializing the reporter facility used for printing well formatted runtime and progress information.

In [ ]:
import os
import sys

verbose  = os.environ.get('RADICAL_PILOT_VERBOSE', 'REPORT')
os.environ['RADICAL_PILOT_VERBOSE'] = verbose

import radical.pilot as rp
import radical.utils as ru
# we use a reporter class for nicer output
report = ru.Reporter(name='radical.pilot')
report.title('Getting Started (RP version %s)' % rp.version)


We will set the resource value to 'local.localhost'. Using a resource key other than local.localhost implicitly tells RADICAL-Pilot that it is targeting a remote resource.

In [ ]:
resource = 'local.localhost'


To create a new Session, you need to provide the URL of a MongoDB server which we will fetch from our .env file.
Create a new session. No need to try/except this: if session creation
fails, there is not much we can do anyways...

In [ ]:
session = rp.Session()

All other pilot code is now tried/excepted. If an exception is caught, we can rely on the session object to exist and be valid, and we can thus tear the whole RP stack down via a <i>'session.close()'</i> call in the <i>'finally'</i> clause.

In [ ]:
def create_pilot_descriptions(resources):
    report.info('read config')
    config = ru.read_json('../config.json')

#     config = ru.read_json('%s/config.json' % os.path.dirname(os.path.abspath(__file__)))
    report.ok('>>ok\n')

    report.header('submit pilots')

    n = 1
    pdescs = list()
    for resource in resources:

        # Define an [n]-core local pilot that runs for [x] minutes
        # Here we use a dict to initialize the description object
        for i in range(n):
            pd_init = {
                  'resource'      : resource,
                  'runtime'       : 60,   # pilot runtime (min)
                  'exit_on_error' : True,
                  'project'       : config[resource].get('project', None),
                  'queue'         : config[resource].get('queue', None),
                  'access_schema' : config[resource].get('schema', None),
                  'cores'         : config[resource].get('cores', 1),
                  'gpus'          : config[resource].get('gpus', 0),
            }
            pdesc = rp.PilotDescription(pd_init)
            pdescs.append(pdesc)
    return pdescs


 Add a PilotManager. PilotManagers manage one or more pilots.

In [ ]:
def launch_pilots(session,pdesc):
    pmgr = rp.PilotManager(session=session)
    pilots = pmgr.submit_pilots(pdesc)
    return pilots    

In [ ]:
def submit_tasks(pilots):
    report.header('submit tasks')


    tmgr = rp.TaskManager(session=session)
    tmgr.add_pilots(pilots)

    # Each task runs '/bin/date'.
    n = 128  # number of tasks to run
    report.info('create %d task description(s)\n\t' % n)

    tds = list()
    for i in range(0, n):

        td = rp.TaskDescription()
        if i % 10:
            td.executable = '/bin/date'
        else:
            # trigger an error now and then
            td.executable = '/bin/data'  # does not exist
        tds.append(td)
        report.progress()

    report.ok('>>ok\n')

    tasks = tmgr.submit_tasks(tds)

    report.header('gather results')
    tmgr.wait_tasks()
    return tasks

We create the <i>report_task_progress</i> function to report the task status of each task

In [ ]:
def report_task_progress(tasks):
    report.info('\n')
    for task in tasks:
        if task.state in [rp.FAILED, rp.CANCELED]:
            report.plain('  * %s: %s, exit: %5s, err: -%35s-'
                        % (task.uid, task.state[:4],
                           task.exit_code, task.stderr))
            report.error('>>err\n')

        else:
            report.plain('  * %s: %s, exit: %5s, out: %35s'
                        % (task.uid, task.state[:4],
                           task.exit_code, task.stdout))
            report.ok('>>ok\n')

We put all function calls inside a try except block.  Finally, always clean up the session no matter if we caught an exception or not. This will kill all the remaining pilots.

In [ ]:
try:
    pdesc = create_pilot_descriptions(['local.localhost'])
    pilots = launch_pilots(session,pdesc)
    tasks = submit_tasks(pilots)
    report_task_progress(tasks)
except Exception as e:
    # Something unexpected happened in the pilot code above
    report.error('caught Exception: %s\n' % e)
    raise

except (KeyboardInterrupt, SystemExit):
    # the callback called sys.exit(), and we can here catch the
    # corresponding KeyboardInterrupt exception for shutdown.  We also catch
    # SystemExit (which gets raised if the main threads exits for some other
    # reason).
    report.warn('exit requested\n')
finally:
    # always clean up the session, no matter if we caught an exception or
    # not.  This will kill all remaining pilots.
    report.header('finalize')
    session.close()
report.header()